In [1]:
from bastionlab import Connection, Identity
import polars as pl

In [2]:
data_owner = Identity.create("data_owner")
client = Connection("localhost").client

In [4]:
target_names = ["setosa", "versicolor", "virginica"]
df = pl.read_csv(
    "iris.csv",
    has_header=False,
    new_columns=[
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)",
        "target",
    ],
)


df = df.with_column(
    pl.col("target")
    .map(lambda a: a.apply(lambda b: True if target_names[b] == "virginica" else False))
    .alias("target_class")
)

target_class_df = pl.DataFrame({"classes": target_names})
target_class_df = target_class_df.with_row_count("id")

In [5]:
rdf = client.polars.send_df(df)
classes_rdf = client.polars.send_df(target_class_df)

train_rdf = rdf.select(pl.col(["petal length (cm)", "petal width (cm)"])).collect()
test_rdf = rdf.select(pl.col("target_class")).collect()
print(train_rdf, test_rdf)

FetchableLazyFrame(identifier=c51ee0c0-b01f-45c9-8ca6-45423c534633) FetchableLazyFrame(identifier=bf51a7ec-80a6-495c-8b5c-722830d81b8f)


In [6]:
from bastionlab.linfa.trainers import LogisticRegression

trainer = client.linfa.train(train_rdf, test_rdf, LogisticRegression())

In [7]:
res = client.linfa.predict(trainer, [5.5, 4.2])
print(res)

AttributeError: 'Client' object has no attribute 'predict'

In [ ]:
res = client.linfa.predict_proba(trainer, [5.5, 4.2])
print(res)